In [1]:
library(text2vec)
library(glmnet)
library(slam)
library(pROC)

Loading required package: Matrix

Loaded glmnet 4.1-4

Type 'citation("pROC")' for a citation.


Attaching package: 'pROC'


The following objects are masked from 'package:stats':

    cov, smooth, var




### Split generation

In [2]:
data <- read.table("alldata.tsv", stringsAsFactors = FALSE,
                  header = TRUE)  
testIDs <- read.csv("project3_splits.csv", header = TRUE)
for(j in 1:5){
  dir.create(paste("split_", j, sep=""))
  train <- data[-testIDs[,j], c("id", "sentiment", "review") ]
  test <- data[testIDs[,j], c("id", "review")]
  test.y <- data[testIDs[,j], c("id", "sentiment", "score")]
  
  tmp_file_name <- paste("split_", j, "/", "train.tsv", sep="")
  write.table(train, file=tmp_file_name, 
              quote=TRUE, 
              row.names = FALSE,
              sep='\t')
  tmp_file_name <- paste("split_", j, "/", "test.tsv", sep="")
  write.table(test, file=tmp_file_name, 
              quote=TRUE, 
              row.names = FALSE,
              sep='\t')
  tmp_file_name <- paste("split_", j, "/", "test_y.tsv", sep="")
  write.table(test.y, file=tmp_file_name, 
            quote=TRUE, 
            row.names = FALSE,
            sep='\t')
}

Warning message in dir.create(paste("split_", j, sep = "")):
"'split_1' already exists"
Warning message in dir.create(paste("split_", j, sep = "")):
"'split_2' already exists"
Warning message in dir.create(paste("split_", j, sep = "")):
"'split_3' already exists"
Warning message in dir.create(paste("split_", j, sep = "")):
"'split_4' already exists"
Warning message in dir.create(paste("split_", j, sep = "")):
"'split_5' already exists"


### Generate Initial document term matrix

In [3]:
# load all data and clean the html tags.
train = read.table("alldata.tsv",
                   stringsAsFactors = FALSE,
                   header = TRUE)
train$review = gsub('<.*?>', ' ', train$review)

# construct DT (DocumentTerm) matrix (maximum 4-grams).
stop_words = c("i", "me", "my", "myself", 
               "we", "our", "ours", "ourselves", 
               "you", "your", "yours", 
               "their", "they", "his", "her", 
               "she", "he", "a", "an", "and",
               "is", "was", "are", "were", 
               "him", "himself", "has", "have", 
               "it", "its", "the", "us")
it_train = itoken(train$review,
                  preprocessor = tolower, 
                  tokenizer = word_tokenizer)
tmp.vocab = create_vocabulary(it_train, 
                              stopwords = stop_words, 
                              ngram = c(1L,4L))
tmp.vocab = prune_vocabulary(tmp.vocab, term_count_min = 10,
                             doc_proportion_max = 0.5,
                             doc_proportion_min = 0.001)
dtm_train  = create_dtm(it_train, vocab_vectorizer(tmp.vocab))

as(<dgTMatrix>, "dgCMatrix") is deprecated since Matrix 1.5-0; do as(., "CsparseMatrix") instead



In [4]:
# trim the vocabulary size to 2K using t-test
v.size = dim(dtm_train)[2]
ytrain = train$sentiment

summ = matrix(0, nrow=v.size, ncol=4)
summ[,1] = colapply_simple_triplet_matrix(
  as.simple_triplet_matrix(dtm_train[ytrain==1, ]), mean)
summ[,2] = colapply_simple_triplet_matrix(
  as.simple_triplet_matrix(dtm_train[ytrain==1, ]), var)
summ[,3] = colapply_simple_triplet_matrix(
  as.simple_triplet_matrix(dtm_train[ytrain==0, ]), mean)
summ[,4] = colapply_simple_triplet_matrix(
  as.simple_triplet_matrix(dtm_train[ytrain==0, ]), var)

n1 = sum(ytrain); 
n = length(ytrain)
n0 = n - n1

myp = (summ[,1] - summ[,3])/
  sqrt(summ[,2]/n1 + summ[,4]/n0)

words = colnames(dtm_train)
id = order(abs(myp), decreasing=TRUE)[1:2000]
pos.list = words[id[myp[id]>0]]
neg.list = words[id[myp[id]<0]]

In [5]:
words[id]

[1] "bad"                          "worst"                       
   [3] "great"                        "waste"                       
   [5] "awful"                        "excellent"                   
   [7] "terrible"                     "worse"                       
   [9] "boring"                       "wonderful"                   
  [11] "best"                         "no"                          
  [13] "stupid"                       "of_best"                     
  [15] "nothing"                      "waste_of"                    
  [17] "poor"                         "horrible"                    
  [19] "one_of_best"                  "of_worst"                    
  [21] "minutes"                      "even"                        
  [23] "so_bad"                       "just"                        
  [25] "love"                         "crap"                        
  [27] "supposed"                     "one_of_worst"                
  [29] "acting"                       "poorly"                      
  [31] "perfect"                      "supposed_to"                 
  [33] "at_all"                       "why"                         
  [35] "plot"                         "loved"                       
  [37] "script"                       "ridiculous"                  
  [39] "waste_of_time"                "amazing"                     
  [41] "lame"                         "beautiful"                   
  [43] "worst_movie"                  "avoid"                       
  [45] "superb"                       "well"                        
  [47] "favorite"                     "wasted"                      
  [49] "brilliant"                    "not_even"                    
  [51] "annoying"                     "don't"                       
  [53] "oh"                           "waste_time"                  
  [55] "dull"                         "mess"                        
  [57] "pointless"                    "money"                       
  [59] "supposed_to_be"               "laughable"                   
  [61] "any"                          "highly"                      
  [63] "cheap"                        "life"                        
  [65] "pathetic"                     "must_see"                    
  [67] "also"                         "least"                       
  [69] "thing"                        "bad_acting"                  
  [71] "badly"                        "fantastic"                   
  [73] "unless"                       "very"                        
  [75] "instead"                      "beautifully"                 
  [77] "one_of"                       "if"                          
  [79] "don't_waste"                  "could"                       
  [81] "at_least"                     "1"                           
  [83] "fails"                        "2"                           
  [85] "redeeming"                    "garbage"                     
  [87] "both"                         "today"                       
  [89] "make"                         "only"                        
  [91] "always"                       "couldn't"                    
  [93] "very_well"                    "performance"                 
  [95] "enjoyed"                      "really_bad"                  
  [97] "bad_movie"                    "performances"                
  [99] "than_this"                    "anything"                    
 [101] "how_bad"                      "well_worth"                  
 [103] "reason"                       "avoid_this"                  
 [105] "or"                           "sorry"                       
 [107] "save"                         "worst_movies"                
 [109] "didn't"                       "highly_recommend"            
 [111] "minutes_of"                   "years"                       
 [113] "looks_like"                   "this_great"                  
 [115] "off"                          "8_10"                        
 

In [6]:
vectorizer = vocab_vectorizer(create_vocabulary(words, 
                                                ngram = c(1L, 2L)))
dtm_reduced = create_dtm(it_train, vectorizer)

In [7]:
# use Lasso (with logistic regression) to trim the vocabulary size to 2K.
set.seed(7568)
tmpfit = glmnet(x = dtm_reduced, 
                y = train$sentiment, 
                alpha = 0.05,
                family='binomial')
tmpfit$df

[1]     0     1     2     3     4     5     7    14    18    26    38    49
 [13]    60    75   100   126   152   177   208   242   288   330   371   419
 [25]   470   528   584   674   754   855   935  1052  1187  1321  1472  1645
 [37]  1838  2076  2334  2612  2891  3238  3583  3951  4336  4779  5259  5727
 [49]  6186  6643  7099  7612  8112  8611  9082  9564 10027 10477 10908 11343
 [61] 11763 12196 12600 12945 13316 13662 14041 14359 14686 14984 15262 15532
 [73] 15780 16032 16252 16466 16686 16887 17057 17222 17396 17546 17683 17832
 [85] 17978 18111 18235 18352 18506 18628 18726 18842 18942 19052 19158 19256
 [97] 19347 19434 19519 19601

In [8]:
tmpfit$df[31]

[1] 935

In [9]:
# pick the largest df among the beta values thar are less than 1K 
#(which turns out to be the 31st column), and store the corresponding words in myvocab
myvocab = colnames(dtm_reduced)[which(tmpfit$beta[, 31] != 0)]

In [10]:
vectorizer = vocab_vectorizer(create_vocabulary(myvocab, 
                                                  ngram = c(1L, 2L)))
dtm_test = create_dtm(it_train, vectorizer)

In [11]:
testfit = glmnet(x = dtm_test, 
                 y = train$sentiment, 
                 alpha = 0.05,
                 family='binomial')

In [12]:
pred = predict(testfit, newx=dtm_test, s=0.01)

In [13]:
roc_obj <- roc(train$sentiment, c(pred))
pROC::auc(roc_obj)

Setting levels: control = 0, case = 1

Setting direction: controls < cases



Area under the curve: 0.9715

In [14]:
# Write vocab to text file
some.strs <- c(myvocab)
write.table(some.strs, file = "myvocab.txt",
            quote = FALSE,
            row.names = FALSE,
            col.names = FALSE,
            sep = "\n")

### Submitted code

In [15]:
# This cell is appended to the top of the submitted code
library(text2vec)
library(glmnet)
myvocab <- scan(file = "myvocab.txt", what = character())

In [16]:
# this loop is for our purposes, we will not submit it
scores = {}
for (j in 1:5) {
    start = Sys.time()
    setwd(paste("split_", j, sep=""))
    # below this comment is submitted code unless otherwise stated
    train = read.table("train.tsv",
                       stringsAsFactors = FALSE,
                       header = TRUE)
    train$review <- gsub('<.*?>', ' ', train$review)

    # Create matrix corresponding to vocab
    it_train = itoken(train$review,
                        preprocessor = tolower, 
                        tokenizer = word_tokenizer)
    vectorizer = vocab_vectorizer(create_vocabulary(myvocab, 
                                                      ngram = c(1L, 2L)))
    dtm_train = create_dtm(it_train, vectorizer)
    set.seed(7568)
    fit = glmnet(x = dtm_train, 
                    y = train$sentiment, 
                    alpha = 0.05,
                    family='binomial')
    
    #####################################
    # Load test data, and 
    # Compute prediction
    #####################################
    test <- read.table("test.tsv", stringsAsFactors = FALSE,
                    header = TRUE)

    test$review <- gsub('<.*?>', ' ', test$review)

    # Create matrix corresponding to vocab
    it_test = itoken(test$review,
                        preprocessor = tolower, 
                        tokenizer = word_tokenizer)
    vectorizer = vocab_vectorizer(create_vocabulary(myvocab, 
                                                ngram = c(1L, 2L)))
    dtm_test = create_dtm(it_test, vectorizer)
    
    prob = predict(fit, newx=dtm_test, s=0.05)
    output = data.frame(id=c(test$id),
                        prob=c(prob))
    #####################################
    # Store your prediction for test data in a data frame
    # "output": col 1 is test$id
    #           col 2 is the predicted probs
    #####################################
    write.table(output, file = "mysubmission.txt", 
                row.names = FALSE, sep='\t')
    
    # Below this is not submitted code
    # move "test_y.tsv" to this directory
    test.y <- read.table("test_y.tsv", header = TRUE)
    pred <- read.table("mysubmission.txt", header = TRUE)
    pred <- merge(pred, test.y, by="id")
    roc_obj <- roc(pred$sentiment, pred$prob)
    score = pROC::auc(roc_obj)
    scores[j] = score
    setwd('../')
    print(Sys.time() - start)
    }

Setting levels: control = 0, case = 1

Setting direction: controls < cases



Time difference of 15.95507 secs


Setting levels: control = 0, case = 1

Setting direction: controls < cases



Time difference of 15.39295 secs


Setting levels: control = 0, case = 1

Setting direction: controls < cases



Time difference of 18.32962 secs


Setting levels: control = 0, case = 1

Setting direction: controls < cases



Time difference of 15.38866 secs


Setting levels: control = 0, case = 1

Setting direction: controls < cases



Time difference of 15.42422 secs


In [17]:
scores

[1] 0.9640572 0.9635631 0.9634647 0.9640653 0.9633994